In [11]:
from dotenv import load_dotenv
import os 

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [13]:
main_word = 'galvanize'

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0)

In [223]:
template_sys = """You are a bot who loves to help people to learn new words. You main adjective is to help people to memorize words. Repeat word.
Classify the level of the word either it is a1,a2,b1,b2,c1,c2 vocabulary and add up either it is a formal, informal, business, slang vocabulary
You should give a list that consists of one or a few meanings if the word has several meanings. Add examples as a list of lists, put in every inner list 5 examples of according meaning. 
Do not forget to add up antonyms and synonims of the word if there any. 
Output should be a valid json. It should contain following fields: word, level, classification, meaning, examples, synonyms, antonyms. If you don't have values in fields synonyms or antonyms feel free to skip them"""


prompt_sys = SystemMessagePromptTemplate.from_template(template_sys)

template_user = """{word}"""

prompt_user = HumanMessagePromptTemplate.from_template(template_user)

chat_prompt = ChatPromptTemplate.from_messages([prompt_sys, prompt_user])

In [225]:
res = chat(chat_prompt.format_messages(word='noun'))

In [226]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List 

In [227]:
class Word(BaseModel):
    word: str = Field(description='the word itself')
    level: str = Field(description='level of vocabulary')
    classification: str = Field(description='either the word is formal, informal or business')
    meaning: List[str] = Field(description='meaning of the word')
    synonyms: List[str] = []
    antonyms: List[str] = []
    examples: List[List] = Field(description='list of examples of the usage of the word')

parser = PydanticOutputParser(pydantic_object=Word)

In [228]:
print(res.content)

{
  "word": "noun",
  "level": "A1",
  "classification": "Formal",
  "meaning": [
    "A word that represents a person, place, thing, or idea",
    "A part of speech that typically functions as the subject or object of a sentence"
  ],
  "examples": [
    ["Cat", "Dog", "House", "Love", "Friend"],
    ["The cat is sleeping.", "I saw a dog in the park.", "The house is big.", "Love is a beautiful feeling.", "My friend is coming over."]
  ],
  "synonyms": [],
  "antonyms": []
}


In [229]:
output = parser.parse(res.content)

In [230]:
output

Word(word='noun', level='A1', classification='Formal', meaning=['A word that represents a person, place, thing, or idea', 'A part of speech that typically functions as the subject or object of a sentence'], synonyms=[], antonyms=[], examples=[['Cat', 'Dog', 'House', 'Love', 'Friend'], ['The cat is sleeping.', 'I saw a dog in the park.', 'The house is big.', 'Love is a beautiful feeling.', 'My friend is coming over.']])